In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.layers import Input, Flatten, Dense
from tensorflow.python.keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False) # data_format = "channels_first"

# Freeze the layers except the last 4 layers
for layer in model_vgg16_conv.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in model_vgg16_conv.layers:
    print(layer, layer.trainable)

#Create your own input format (here 3x200x200)
input = Input(shape=(150,150,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = Dense(1, activation='linear', name='pred')(x)

#Create your own model 
my_model = Model(input, x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training

my_model.compile(loss='mse',
              optimizer='adam',
              metrics=['mse'])
my_model.summary()

58892288/58889256 [==============================] - 1s 0us/step
<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f0fc9af0940> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0fc9ae8eb8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0fc9a997b8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0fc925b9b0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0fc925b828> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0fc926d898> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0fc92742b0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0fc9274ba8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0fc927eeb8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0fc0440470> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0fc04454a8> Fals

In [0]:
# load data into dataframe with 'path' and 'target' columns
import pandas as pd
train_label_df = pd.read_csv('/content/drive/My Drive/team7/data.csv', delimiter=' ', header=1, names=["uid"])
train_label_df["path"] = train_label_df['uid'].apply(lambda x : '/content/drive/My Drive/team7/data/' + x.split(',')[-1][1:].split('/')[-1])
train_label_df["target"] = train_label_df['uid'].apply(lambda x : int(x.split(',')[1])/int(x.split(',')[2]))
train_label_df =train_label_df.drop(['uid'], axis=1)

train_label_df2 = pd.read_csv('/content/drive/My Drive/team7/data_temp.csv', delimiter=' ', header=1, names=["uid"])
train_label_df2["path"] = train_label_df2['uid'].apply(lambda x : "/content/drive/My Drive/team7/data/" + x.split(',')[0] + ".jpg")
train_label_df2["target"] = train_label_df2['uid'].apply(lambda x : int(x.split(',')[1])/int(x.split(',')[3]))
train_label_df2 =train_label_df2.drop(['uid'], axis=1)
#uid,likes,comments,followers
#B-A-bW9l1Gm,601,43,6715

In [0]:
train_df = pd.concat([train_label_df, train_label_df2], axis=0).reset_index().drop(['index'], axis=1)

In [0]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('/content/drive/My Drive/team7/data/') if isfile(join('/content/drive/My Drive/team7/data/', f))]

In [7]:
new_df = train_df
j = 0
for i in train_df['path']:
    #print(i.split("/")[-1])
    if i.split("/")[-1] not in onlyfiles:
        new_df = new_df.drop(train_df.index[train_df['path'] == i].tolist())
    else:
      j+=1
train_df = new_df.reset_index().drop(['index'], axis=1)
# All are found
train_df

,path,target
0,/content/drive/My Drive/team7/data/B9dZzI1n-EV...,0.112025
1,/content/drive/My Drive/team7/data/B9cQMfVnkMe...,0.638448
2,/content/drive/My Drive/team7/data/B9a0IQWnY9M...,0.053981
3,/content/drive/My Drive/team7/data/B9NnTUrHoNX...,0.143885
4,/content/drive/My Drive/team7/data/B9XuP83gx_j...,0.030143
...,...,...
2030,/content/drive/My Drive/team7/data/B_UgmnQj0tQ...,0.041773
2031,/content/drive/My Drive/team7/data/B_QoOOxl-ve...,0.024106
2032,/content/drive/My Drive/team7/data/B_OM7wVoCUd...,0.025871
2033,/content/drive/My Drive/team7/data/B_QTlktFkRZ...,0.014322


In [0]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(train_df, test_size=0.1)

In [9]:
batch_size = 128
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator()

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator()

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,
        x_col = 'path',
        y_col = 'target',
        # directory='data',
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='other',
       #  preprocessing_function=lambda x: x print(x)
)  # since we use binary_crossentropy loss, we need binary labels

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test,
        x_col = 'path',
        y_col = 'target',
        # directory='data',
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='other',
)

Found 1831 validated image filenames.
Found 204 validated image filenames.


In [10]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())  

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']


In [11]:
my_model.fit_generator(
        train_generator,
        steps_per_epoch=1280,
        epochs=5,
        #validation_data=validation_generator,
        #validation_steps=800 // batch_size)
        verbose = 2,
        # use_multiprocessing=False
)
my_model.save_weights('first_try.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
1280/1280 - 2331s - loss: 44.8555 - mse: 47.0055
Epoch 2/5
1280/1280 - 1552s - loss: 0.3423 - mse: 0.3486
Epoch 3/5
1280/1280 - 1550s - loss: 0.3342 - mse: 0.3416
Epoch 4/5
1280/1280 - 1556s - loss: 0.3347 - mse: 0.3412
Epoch 5/5
1280/1280 - 1567s - loss: 0.3376 - mse: 0.3439


In [17]:
my_model.evaluate(test_generator, steps=400)

400/400 [==============================] - 401s 1s/step - loss: 0.0205 - mse: 0.0206


[0.02051091007888317, 0.02059905044734478]